<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/3_Analisis_y_visualizacion_datos/4_Manipulacion_y_Transformacion_de_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wbdata

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# 🪄**Manipulación y transformación de datos**

👉 Ahora que los datos están limpios, toca organizarlos para analizarlos.

✅ Resultado: un dataset estructurado para responder preguntas económicas.

**Objetivos de Aprendizaje:**

Al finalizar este notebook, serás capaz de:

1. **Organizar y filtrar datos** para aislar la información más relevante para una pregunta de investigación económica.

2. **Agregar y resumir datos** para calcular estadísticas descriptivas a nivel de grupo (e.g., por región, por nivel de ingreso).

3. **Remodelar y transformar tablas de datos** para pasar de formatos anchos a largos y viceversa, una habilidad crucial para preparar datos para modelos econométricos y visualizaciones.

**Introducción**

Imagina que eres un economista recién contratado en el Banco Mundial. Tu primer encargo es analizar la relación entre el Gasto Público en Educación (% del PIB) y el Crecimiento del PIB per cápita para diferentes regiones del mundo durante la última década.

Recibes un enorme archivo csv con miles de filas y columnas, mezclando países, años e indicadores de todo tipo. No puedes simplemente introducir eso en un modelo. Primero, necesitas esculpir y analizar tus datos. Necesitas:

- Ordenar los países para ver los que más y menos invierten.

- Filtrar para quedarte solo con los años y los indicadores relevantes.

- Agrupar por región para calcular el gasto promedio y ver si África Subsahariana invierte, en promedio, más o menos que América Latina.

- Pivotar la tabla para que cada fila sea un país y cada columna un año, permitiéndote ver la evolución de un vistazo.

Estas no son solo "funciones de pandas"; son las herramientas diarias de un economista.

## Importar librerías

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Mejorar visualización de los dataframes

In [3]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

## Cargar el dataset

Como ejemplo, usaremos un **dataset real del Banco Mundial** que contiene indicadores de desarrollo. Esto nos permitirá realizar un análisis que un economista haría en su día a día.

Utilizaremos la API del Banco Mundial para obtener los datos.

In [4]:
import wbdata

In [5]:
# Listar bases de datos disponibles en el Banco Mundial
# Nos interesa World Development Indicators
wbdata.get_sources()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  25  Jobs
  27  Global Economic Prospects
  28  Global Findex database
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclusion Indicators
  34  Global P

In [6]:
# Listar indicadores disponibles en World Development Indicators
wbdata.get_indicators(source=2)

id                          name
--------------------------  ---------------------------------------------------------------------------------------------------------------------------------------------
AG.CON.FERT.PT.ZS           Fertilizer consumption (% of fertilizer production)
AG.CON.FERT.ZS              Fertilizer consumption (kilograms per hectare of arable land)
AG.LND.AGRI.K2              Agricultural land (sq. km)
AG.LND.AGRI.ZS              Agricultural land (% of land area)
AG.LND.ARBL.HA              Arable land (hectares)
AG.LND.ARBL.HA.PC           Arable land (hectares per person)
AG.LND.ARBL.ZS              Arable land (% of land area)
AG.LND.CREL.HA              Land under cereal production (hectares)
AG.LND.CROP.ZS              Permanent cropland (% of land area)
AG.LND.EL5M.RU.K2           Rural land area where elevation is below 5 meters (sq. km)
AG.LND.EL5M.RU.ZS           Rural land area where elevation is below 5 meters (% of total land area)
AG.LND.EL5M.UR.K2  

In [7]:
# Buscar los indicadores de interés en World Development Indicators
wbdata.get_indicators(query="GDP per capita", source=2)

id                 name
-----------------  -------------------------------------------------------------------
NY.GDP.PCAP.CD     GDP per capita (current US$)
NY.GDP.PCAP.CN     GDP per capita (current LCU)
NY.GDP.PCAP.KD     GDP per capita (constant 2015 US$)
NY.GDP.PCAP.KD.ZG  GDP per capita growth (annual %)
NY.GDP.PCAP.KN     GDP per capita (constant LCU)
NY.GDP.PCAP.PP.CD  GDP per capita, PPP (current international $)
NY.GDP.PCAP.PP.KD  GDP per capita, PPP (constant 2021 international $)
SE.XPD.PRIM.PC.ZS  Government expenditure per student, primary (% of GDP per capita)
SE.XPD.SECO.PC.ZS  Government expenditure per student, secondary (% of GDP per capita)
SE.XPD.TERT.PC.ZS  Government expenditure per student, tertiary (% of GDP per capita)

In [9]:
indicadores = {"NY.GDP.PCAP.PP.CD": "PIB per cápita (PPA, USD internacionales)",
               "SH.XPD.CHEX.PP.CD": "Gasto en salud per cápita (PPA, USD internacionales)",
               "SP.DYN.LE00.IN": "Esperanza de vida (años)",
               "SL.UEM.TOTL.NE.ZS": "Tasa de desempleo (%)"}

# Descargar datos para países de América del Sur de 2010 a 2022
df = wbdata.get_dataframe(indicadores, country=["COL", "PER", "BRA", "CHL", "ARG", "BOL", "ECU", "PRY", "URY"], date=("2010", "2022"))
df = df.reset_index()
df = df.rename(columns={'country': 'País', 'date': 'Año'})

## Exploración inicial del dataset

In [10]:
df

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
1,Argentina,2021,"26,300.27","2,418.80",73.95,8.74
2,Argentina,2020,"22,393.35","2,182.31",75.88,11.46
3,Argentina,2019,"23,516.83","2,334.76",76.85,9.84
4,Argentina,2018,"24,410.39","2,388.69",76.77,9.22
...,...,...,...,...,...,...
112,Uruguay,2014,"21,867.81","1,785.41",77.19,6.55
113,Uruguay,2013,"20,661.38","1,709.51",76.95,6.44
114,Uruguay,2012,"19,495.42","1,615.23",76.69,6.45
115,Uruguay,2011,"19,135.36","1,574.28",76.61,6.31


In [11]:
df.shape

(117, 6)

In [12]:
df.info()

<class 'wbdata.client.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 6 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   País                                                  117 non-null    object 
 1   Año                                                   117 non-null    object 
 2   PIB per cápita (PPA, USD internacionales)             117 non-null    float64
 3   Gasto en salud per cápita (PPA, USD internacionales)  117 non-null    float64
 4   Esperanza de vida (años)                              117 non-null    float64
 5   Tasa de desempleo (%)                                 113 non-null    float64
dtypes: float64(4), object(2)
memory usage: 5.6+ KB


In [13]:
df["País"].value_counts(dropna=False) # este me realiza un conteo por país el dropna es para que ignore el na

,count
País,
Argentina,13
Bolivia,13
Brazil,13
Chile,13
Colombia,13
Ecuador,13
Peru,13
Paraguay,13
Uruguay,13


Tenemos una base de datos con información de 9 países de América del Sur para 13 años (2010-2022), lo que nos da un total de 117 filas. Se trajo información de 4 indicadores. La información se presenta en una estructura de panel de datos.

## Realizar filtros

En ocasiones, un economista quiere seleccionar subconjuntos de datos para analizar. Por ejemplo, podemos estar interesados en conocer cómo se comportaron los indicadores en los distintos países en el año 2022, por lo que vamos a crear un dataframe solo con la información de 2022.

In [15]:
df_2022 = df[df['Año'] == "2022"]
df_2022
#Sirve para mirar unicamente el año 2022

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
13,Bolivia,2022,"10,371.96",830.75,67.43,3.55
26,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
39,Chile,2022,"30,820.35","3,171.26",79.18,8.25
52,Colombia,2022,"20,695.74","1,598.87",76.51,10.55
65,Ecuador,2022,"15,198.07",989.05,76.58,3.71
78,Peru,2022,"16,669.16",941.82,76.83,4.29
91,Paraguay,2022,"16,347.04","1,247.31",72.32,6.75
104,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88


from matplotlib import pyplot as plt
df_2022['PIB per cápita (PPA, USD internacionales)'].plot(kind='hist', bins=20, title='PIB per cápita (PPA, USD internacionales)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_2022['Gasto en salud per cápita (PPA, USD internacionales)'].plot(kind='hist', bins=20, title='Gasto en salud per cápita (PPA, USD internacionales)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_2022['Esperanza de vida (años)'].plot(kind='hist', bins=20, title='Esperanza de vida (años)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_2022['Tasa de desempleo (%)'].plot(kind='hist', bins=20, title='Tasa de desempleo (%)')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
df_2022.plot(kind='scatter', x='Esperanza de vida (años)', y='Tasa de desempleo (%)', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_2022['PIB per cápita (PPA, USD internacionales)'].plot(kind='line', figsize=(8, 4), title='PIB per cápita (PPA, USD internacionales)')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_2022['Gasto en salud per cápita (PPA, USD internacionales)'].plot(kind='line', figsize=(8, 4), title='Gasto en salud per cápita (PPA, USD internacionales)')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_2022['Esperanza de vida (años)'].plot(kind='line', figsize=(8, 4), title='Esperanza de vida (años)')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_2022['Tasa de desempleo (%)'].plot(kind='line', figsize=(8, 4), title='Tasa de desempleo (%)')
plt.gca().spines[['top', 'right']].set_visible(False)

In [17]:
# Reiniciamos el índice para que lo roganice y resetee el indice
df_2022.reset_index(drop=True, inplace=True)
df_2022

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
1,Bolivia,2022,"10,371.96",830.75,67.43,3.55
2,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
4,Colombia,2022,"20,695.74","1,598.87",76.51,10.55
5,Ecuador,2022,"15,198.07",989.05,76.58,3.71
6,Peru,2022,"16,669.16",941.82,76.83,4.29
7,Paraguay,2022,"16,347.04","1,247.31",72.32,6.75
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88


### Filtrar por el índice

Queremos ver la información correspondiente al índice 3

In [18]:
df_2022.iloc[3] #sirve para ver unciamente un indice

,3
País,Chile
Año,2022
"PIB per cápita (PPA, USD internacionales)","30,820.35"
"Gasto en salud per cápita (PPA, USD internacionales)","3,171.26"
Esperanza de vida (años),79.18
Tasa de desempleo (%),8.25


### Filtrar por una columna

Queremos ver la información de Colombia

In [23]:
df_2022[df_2022['País']=="Colombia"]
#para filtrar por fila

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
4,Colombia,2022,"20,695.74","1,598.87",76.51,10.55


**Ejercico**

Muéstrame todos los países que sean diferentes a Colombia

In [24]:
df_2022[df_2022['País']!="Colombia"]

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
1,Bolivia,2022,"10,371.96",830.75,67.43,3.55
2,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
5,Ecuador,2022,"15,198.07",989.05,76.58,3.71
6,Peru,2022,"16,669.16",941.82,76.83,4.29
7,Paraguay,2022,"16,347.04","1,247.31",72.32,6.75
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88


**Ejercicio**

¿Qué países tienen una tasa de desempleo menor estrictamente a 5%?

In [26]:
df_2022[df_2022['Tasa de desempleo (%)'] < 5]

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
1,Bolivia,2022,"10,371.96",830.75,67.43,3.55
5,Ecuador,2022,"15,198.07",989.05,76.58,3.71
6,Peru,2022,"16,669.16",941.82,76.83,4.29


### Filtrar por varias columnas

¿Qué países tienen una tasa de desempleo mayor o igual a 6% y una esperanza de vida mayor o igual a 75 años?

In [28]:
df_2022[(df_2022['Tasa de desempleo (%)']>=6) & (df_2022['Esperanza de vida (años)']>=75)]

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
4,Colombia,2022,"20,695.74","1,598.87",76.51,10.55
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88


¿Qué países tienen un PIB per cápita PPA mayor a USD 25.000 o un gasto en salud per cápita PPA mayor a USD 1.000?

In [29]:
df_2022[(df_2022['PIB per cápita (PPA, USD internacionales)']>25000) | (df_2022['Gasto en salud per cápita (PPA, USD internacionales)']>1000)] # el palito en la mitad sirve para hacer la expresion de este o este

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
2,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
4,Colombia,2022,"20,695.74","1,598.87",76.51,10.55
7,Paraguay,2022,"16,347.04","1,247.31",72.32,6.75
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88


### Filtrar por el mínimo o máximo de una columna

¿Cuál país tiene el PIB per cápita más alto?

In [30]:
df_2022[df_2022['PIB per cápita (PPA, USD internacionales)'] == df_2022['PIB per cápita (PPA, USD internacionales)'].max()]

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88


**Ejercicio**

¿Cuál país tiene la esperanza de vida más baja?

In [32]:
df_2022[df_2022['Esperanza de vida (años)'] == df_2022['Esperanza de vida (años)'].min()]

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
1,Bolivia,2022,"10,371.96",830.75,67.43,3.55


**Ejercicio**

¿Qué países tienen un gasto en salud per cápita PPA por encima del promedio?

In [33]:
df_2022[df_2022['Gasto en salud per cápita (PPA, USD internacionales)'] > df_2022['Gasto en salud per cápita (PPA, USD internacionales)'].mean()]

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88


### Filtrar por una lista de valores

Muéstrame los datos para Brasil y Ecuador

In [35]:
df_2022[df_2022['País'].isin(['Brazil','Ecuador'])]
#filtrar por las categorias o paies que quiero que filtre

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
2,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
5,Ecuador,2022,"15,198.07",989.05,76.58,3.71


### Filtrar los strings que contengan ciertos caracteres

Muéstrame los países que contengan "P"

In [ ]:
df_2022[df_2022['País'].str.contains("P", na=False)]

Muéstrame los países que contengan "Pe" o "Br" o "Ch"

In [37]:
df_2022[df_2022['País'].str.contains(r'Pe|Br|Ch', na=False)]
#paies o columnas que contengan lo que indico (no olvidar que es sensible a las columans)

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
2,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
6,Peru,2022,"16,669.16",941.82,76.83,4.29


**Ejercicio**

Muestra los países que contengan "guay"

In [38]:
df_2022[df_2022['País'].str.contains(r'guay', na=False)]


,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
7,Paraguay,2022,"16,347.04","1,247.31",72.32,6.75
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88


### Filtrar por negación (todo excepto ...)

Muéstrame todos los países excepto Brasil

In [39]:
df_2022[~(df_2022['País']=='Brazil')] #

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
1,Bolivia,2022,"10,371.96",830.75,67.43,3.55
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
4,Colombia,2022,"20,695.74","1,598.87",76.51,10.55
5,Ecuador,2022,"15,198.07",989.05,76.58,3.71
6,Peru,2022,"16,669.16",941.82,76.83,4.29
7,Paraguay,2022,"16,347.04","1,247.31",72.32,6.75
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88


**Ejercicio**

Muéstrame todos los países excepto los que terminan en "guay". Filtra utilizando negación.

In [41]:
df_2022[~df_2022['País'].str.endswith('guay')]

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
1,Bolivia,2022,"10,371.96",830.75,67.43,3.55
2,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
4,Colombia,2022,"20,695.74","1,598.87",76.51,10.55
5,Ecuador,2022,"15,198.07",989.05,76.58,3.71
6,Peru,2022,"16,669.16",941.82,76.83,4.29


**Ejercicio**

Muestra los países donde su PIB per cápita no sea mayor a USD 25.000 PPA y su tasa de desempleo no sea menor a 5%. Filtra utilizando negación.

In [47]:
df_2022[~(df_2022['PIB per cápita (PPA, USD internacionales)']>25000) & ~df_2022['Tasa de desempleo (%)']<5)]

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (ipython-input-2853181399.py, line 1)

## Ordenar el dataframe

Para un economista, ordenar es sinónimo de ranking. ¿Qué país tiene el mayor PIB? ¿Cuál tiene la menor tasa de inflación? sort_values() nos permite responder a estas preguntas.

### Ordenar según el valor de una columna

Vamos a ordenar los países de menor a mayor gasto en salud per cápita

In [48]:
df_2022.sort_values(by=['Gasto en salud per cápita (PPA, USD internacionales)'], ascending = True) #ascending ees por fecto

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
1,Bolivia,2022,"10,371.96",830.75,67.43,3.55
6,Peru,2022,"16,669.16",941.82,76.83,4.29
5,Ecuador,2022,"15,198.07",989.05,76.58,3.71
7,Paraguay,2022,"16,347.04","1,247.31",72.32,6.75
4,Colombia,2022,"20,695.74","1,598.87",76.51,10.55
2,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25


**Ejercicio**

Ordena los países de mayor a menor PIB per cápita PPA

In [49]:
df_2022.sort_values(by=['Gasto en salud per cápita (PPA, USD internacionales)'], ascending = False)

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
8,Uruguay,2022,"33,000.92","2,568.07",76.47,7.88
2,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
4,Colombia,2022,"20,695.74","1,598.87",76.51,10.55
7,Paraguay,2022,"16,347.04","1,247.31",72.32,6.75
5,Ecuador,2022,"15,198.07",989.05,76.58,3.71
6,Peru,2022,"16,669.16",941.82,76.83,4.29
1,Bolivia,2022,"10,371.96",830.75,67.43,3.55


**Ejercicio**

 Encuentra los 5 países con la menor tasa de desempleo en 2022.

In [56]:
df_2022.sort_values(by=['Tasa de desempleo (%)'], ascending = True)
df_2022.head(5)

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
1,Bolivia,2022,"10,371.96",830.75,67.43,3.55
2,Brazil,2022,"19,876.85","1,696.41",74.87,9.23
3,Chile,2022,"30,820.35","3,171.26",79.18,8.25
4,Colombia,2022,"20,695.74","1,598.87",76.51,10.55


### Ordenar según el valor de varias columnas

In [57]:
# Datos económicos sintéticos de varios países
datos = {
    'País': ['Colombia', 'Chile', 'México', 'Argentina', 'Perú', 'Panamá'],
    'Región': ['Andina', 'Andina', 'Norteamérica', 'Sur', 'Andina', 'Centroamérica'],
    'Crecimiento_PIB_%': [1.5, 2.1, 2.5, -1.1, 2.8, 4.1],
    'Tasa_Desempleo_%': [10.5, 8.8, 3.5, 11.2, 7.5, 8.1]
}

# Creamos el DataFrame
df_ejemplo = pd.DataFrame(datos)

df_ejemplo

,País,Región,Crecimiento_PIB_%,Tasa_Desempleo_%
0,Colombia,Andina,1.50,10.50
1,Chile,Andina,2.10,8.80
2,México,Norteamérica,2.50,3.50
3,Argentina,Sur,-1.10,11.20
4,Perú,Andina,2.80,7.50
5,Panamá,Centroamérica,4.10,8.10


In [58]:
df_ordenado = df_ejemplo.sort_values(by=['Región', 'Crecimiento_PIB_%'], ascending=[True, False])
df_ordenado

,País,Región,Crecimiento_PIB_%,Tasa_Desempleo_%
4,Perú,Andina,2.80,7.50
1,Chile,Andina,2.10,8.80
0,Colombia,Andina,1.50,10.50
5,Panamá,Centroamérica,4.10,8.10
2,México,Norteamérica,2.50,3.50
3,Argentina,Sur,-1.10,11.20


## Groupby

Esta es una de las herramientas más utilizadas, ya que permite agrupar por categorías y sacar mediciones para cada grupo como la media, la suma, la desviación estándar, el conteo de registros, etc.

Para ello, se emplea la estrategia de Dividir-Aplicar-Combinar.

- **Dividir (Split):** Se separa el DataFrame en grupos basados en una o más variables categóricas (ejemplo: País, Año, etc).

- **Aplicar (Apply):** A cada grupo se le aplica una función (ejemplo: mean, sum, std).

- **Combinar (Combine):** Se juntan los resultados en un nuevo DataFrame.

Es como si el FMI quisiera calcular la tasa de inflación promedio para cada continente. Primero, divide el dataset global por continente. Luego, aplica la función de promedio a la columna de inflación para cada grupo. Finalmente, combina los resultados en una tabla resumen.

In [59]:
# Dataframe original
df

,País,Año,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)",Esperanza de vida (años),Tasa de desempleo (%)
0,Argentina,2022,"29,597.69","2,664.35",75.81,6.80
1,Argentina,2021,"26,300.27","2,418.80",73.95,8.74
2,Argentina,2020,"22,393.35","2,182.31",75.88,11.46
3,Argentina,2019,"23,516.83","2,334.76",76.85,9.84
4,Argentina,2018,"24,410.39","2,388.69",76.77,9.22
...,...,...,...,...,...,...
112,Uruguay,2014,"21,867.81","1,785.41",77.19,6.55
113,Uruguay,2013,"20,661.38","1,709.51",76.95,6.44
114,Uruguay,2012,"19,495.42","1,615.23",76.69,6.45
115,Uruguay,2011,"19,135.36","1,574.28",76.61,6.31


¿Cuál fue la esperanza de vida promedio de cada país entre 2010 y 2022?

In [60]:
df.groupby('País')['Esperanza de vida (años)'].mean().reset_index()

,País,Esperanza de vida (años)
0,Argentina,76.01
1,Bolivia,66.31
2,Brazil,74.69
3,Chile,79.69
4,Colombia,75.65
5,Ecuador,75.56
6,Paraguay,72.77
7,Peru,74.79
8,Uruguay,77.05


Ordenemos de mayor a menor...

In [61]:
df.groupby('País')['Esperanza de vida (años)'].mean().reset_index().sort_values(by='Esperanza de vida (años)', ascending=False)

,País,Esperanza de vida (años)
3,Chile,79.69
8,Uruguay,77.05
0,Argentina,76.01
4,Colombia,75.65
5,Ecuador,75.56
7,Peru,74.79
2,Brazil,74.69
6,Paraguay,72.77
1,Bolivia,66.31


¿Cuál fue la tasa de desempleo más baja que registró cada país entre 2010 y 2022?

In [62]:
df.groupby('País')['Tasa de desempleo (%)'].min().reset_index()

,País,Tasa de desempleo (%)
0,Argentina,6.80
1,Bolivia,2.02
2,Brazil,6.75
3,Chile,6.07
4,Colombia,8.57
5,Ecuador,3.08
6,Paraguay,4.09
7,Peru,3.21
8,Uruguay,6.31


¿Cuánto sumó el gasto en salud per cápita de estos países en cada uno de los años de estudio (2010-2022)?

In [63]:
df.groupby('Año')['Gasto en salud per cápita (PPA, USD internacionales)'].sum().reset_index()

,Año,"Gasto en salud per cápita (PPA, USD internacionales)"
0,2010,"8,175.24"
1,2011,"8,774.69"
2,2012,"9,245.96"
3,2013,"9,864.77"
4,2014,"10,339.35"
5,2015,"10,804.54"
6,2016,"11,131.81"
7,2017,"12,108.55"
8,2018,"12,615.04"
9,2019,"13,037.77"


Para cada país de América del Sur, ¿cuál fue el promedio de PIB per cápita PPA y de gasto en salud per cápita PPA entre 2010 y 2022?

In [64]:
df.groupby("País")[["PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)"]].mean().reset_index()

,País,"PIB per cápita (PPA, USD internacionales)","Gasto en salud per cápita (PPA, USD internacionales)"
0,Argentina,"21,954.46","2,127.13"
1,Bolivia,"7,650.02",503.81
2,Brazil,"15,822.21","1,376.50"
3,Chile,"23,872.92","2,057.80"
4,Colombia,"14,427.34","1,102.20"
5,Ecuador,"11,586.79",826.48
6,Paraguay,"13,259.22",896.90
7,Peru,"12,396.53",646.81
8,Uruguay,"23,536.58","1,959.56"


Para cada país de América del Sur, ¿cuál fue el promedio de gasto en salud per cápita PPA y cuál fue la máxima esperanza de vida entre 2010 y 2022?

In [66]:
df.groupby("País", as_index=False).agg(
    gasto_promedio_salud = ("Gasto en salud per cápita (PPA, USD internacionales)", "mean"),
    max_esperanza_de_vida = ("Esperanza de vida (años)", "max")
)
# para realizar dos operaciones

,País,gasto_promedio_salud,max_esperanza_de_vida
0,Argentina,"2,127.13",76.85
1,Bolivia,503.81,67.82
2,Brazil,"1,376.50",75.81
3,Chile,"2,057.80",80.61
4,Colombia,"1,102.20",76.79
5,Ecuador,826.48,77.29
6,Paraguay,896.90,73.67
7,Peru,646.81,76.83
8,Uruguay,"1,959.56",78.38


Para cada país de América del Sur, calcule el mínimo, el máximo y el promedio del gasto en salud per cápita PPA y de la esperanza de vida entre 2010 y 2022

In [67]:
df.groupby("País")[["Gasto en salud per cápita (PPA, USD internacionales)","Esperanza de vida (años)"]].agg(["min", "mean", "max"]).reset_index()

País Gasto en salud per cápita (PPA, USD internacionales)           \
                                                              min     mean   
0  Argentina                                           1,685.55   2,127.13   
1    Bolivia                                             265.31     503.81   
2     Brazil                                           1,148.05   1,376.50   
3      Chile                                           1,221.93   2,057.80   
4   Colombia                                             758.86   1,102.20   
5    Ecuador                                             563.12     826.48   
6   Paraguay                                             595.34     896.90   
7       Peru                                             462.29     646.81   
8    Uruguay                                           1,474.80   1,959.56   

           Esperanza de vida (años)              
       max                      min  mean   max  
0 2,664.35                    73.95 76.01 76.85  
1   830.75                    61.43 66.31 67.82  
2 1,696.41                    73.04 74.69 75.81  
3 3,171.26                    78.88 79.69 80.61  
4 1,598.87                    72.70 75.65 76.79  
5   989.05                    72.00 75.56 77.29  
6 1,247.31                    68.11 72.77 73.67  
7   949.83                    71.60 74.79 76.83  
8 2,568.07                    75.43 77.05 78.38

**Ejercicio**

Para cada país de América del Sur, calcule la mediana y la desviación estándar del PIB per cápita PPA entre 2010 y 2022

In [69]:
df.groupby("País")[["PIB per cápita (PPA, USD internacionales)"]].agg(["median", "std"]).reset_index()

País PIB per cápita (PPA, USD internacionales)         
                                                median      std
0  Argentina                                 20,105.76 3,381.37
1    Bolivia                                  7,547.90 1,642.63
2     Brazil                                 15,463.74 1,563.20
3      Chile                                 23,256.50 3,483.97
4   Colombia                                 14,026.84 2,644.02
5    Ecuador                                 11,526.80 1,611.37
6   Paraguay                                 13,329.14 1,672.52
7       Peru                                 12,066.55 1,932.85
8    Uruguay                                 22,840.55 4,243.44

## Tablas pivote o dinámicas

Una tabla pivote es una forma avanzada de groupby que nos **permite remodelar el DataFrame**, poniendo una variable categórica como índice, otra como columnas y una variable numérica como los valores. **Es similar a las tablas dinámicas en Excel.**

Por ejemplo, creemos una tabla para un informe donde las filas son los países, las columnas son los años (2010, 2011, 2012...), y los valores en las celdas son la tasa de desempleo. Esto facilita enormemente la comparación de la evolución del desempleo entre países.

In [84]:
df_desempleo = df.pivot_table(index='País', columns='Año', values='Tasa de desempleo (%)').reset_index()
df_desempleo

Año,País,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Argentina,7.71,7.18,7.22,7.10,7.27,NaN,NaN,8.35,9.22,9.84,11.46,8.74,6.80
1,Bolivia,NaN,2.25,2.07,2.44,2.02,3.11,3.50,3.65,3.52,3.68,7.90,5.09,3.55
2,Brazil,NaN,7.58,7.25,7.07,6.75,8.54,11.58,12.79,12.33,11.94,13.70,13.16,9.23
3,Chile,8.28,7.28,6.56,6.07,6.48,6.31,6.67,7.26,7.62,7.49,10.93,9.28,8.25
4,Colombia,11.15,10.29,9.96,9.25,8.80,8.57,8.92,9.09,9.36,10.28,15.98,13.90,10.55
5,Ecuador,4.09,3.46,3.23,3.08,3.48,3.62,4.60,3.84,3.53,3.81,6.12,4.50,3.71
6,Paraguay,4.57,4.67,4.09,4.38,5.03,4.56,5.25,6.14,6.22,6.59,7.55,7.31,6.75
7,Peru,3.60,3.48,3.24,3.58,3.21,3.27,3.74,3.69,3.49,3.38,7.18,5.10,4.29
8,Uruguay,7.16,6.31,6.45,6.44,6.55,7.49,7.84,7.89,8.34,8.84,10.41,9.33,7.88


Este formato **ancho (wide)** es muy intuitivo. Podemos comparar fácilmente el desempleo en 2022 de los diferentes países de América del Sur y ver que el de Colombia es el más alto.

También podemos seguir la fila de un país para ver su tendencia, como por ejemplo, ver el comportamiento que ha tenido la tasa de desempleo de Colombia entre 2010 y 2022 y ver que durante la pandemia llegó a un máximo de 16%.


A continuación, vamos a crear una tabla que presenta, para cada país, la evolución 2010-2022 del PIB per cápita PPA y de la tasa de desempleo.

In [70]:
df1 = pd.pivot_table(df, index = 'País', columns = 'Año',
                     values = ['PIB per cápita (PPA, USD internacionales)',
                               'Tasa de desempleo (%)']).reset_index()
df1

País PIB per cápita (PPA, USD internacionales)                      \
Año                                                 2010      2011      2012   
0    Argentina                                 17,847.61 19,104.99 19,430.42   
1      Bolivia                                  5,100.53  5,388.51  5,866.98   
2       Brazil                                 14,451.72 15,211.79 15,198.09   
3        Chile                                 17,917.90 20,121.68 21,366.18   
4     Colombia                                 10,841.13 11,707.46 12,093.08   
5      Ecuador                                  8,968.94  9,768.69 10,244.52   
6     Paraguay                                 10,893.31 11,441.76 10,954.33   
7         Peru                                  9,713.36 10,462.58 10,757.15   
8      Uruguay                                 17,872.54 19,135.36 19,495.42   

                                                                           \
Año      2013      2014      2015      2016      2017      2018      2019   
0   19,928.99 19,487.42 19,899.15 20,105.76 23,385.07 24,410.39 23,516.83   
1    6,589.67  7,047.20  7,152.04  7,547.90  8,510.27  8,952.59  9,174.45   
2   15,722.33 15,827.31 14,821.44 14,309.21 14,559.05 15,463.74 16,069.84   
3   22,201.07 22,553.62 22,457.88 23,256.50 24,295.22 25,305.09 25,611.76   
4   12,779.89 13,355.36 13,332.06 14,026.84 14,400.61 15,239.40 16,181.59   
5   11,295.61 11,835.61 10,877.75 10,881.49 11,793.48 12,186.88 12,543.01   
6   12,137.01 12,607.07 12,679.06 13,329.14 13,977.34 14,139.09 14,149.60   
7   11,290.55 11,517.74 11,600.39 12,066.55 12,591.86 13,078.23 13,563.54   
8   20,661.38 21,867.81 22,168.61 22,840.55 23,606.63 24,386.49 25,782.69   

                                  Tasa de desempleo (%)                       \
Año      2020      2021      2022                  2010  2011 2012 2013 2014   
0   22,393.35 26,300.27 29,597.69                  7.71  7.18 7.22 7.10 7.27   
1    8,293.99  9,454.20 10,371.96                   NaN  2.25 2.07 2.44 2.02   
2   16,101.62 18,075.71 19,876.85                   NaN  7.58 7.25 7.07 6.75   
3   25,350.86 29,089.85 30,820.35                  8.28  7.28 6.56 6.07 6.48   
4   15,519.21 17,383.09 20,695.74                 11.15 10.29 9.96 9.25 8.80   
5   11,526.80 13,507.44 15,198.07                  4.09  3.46 3.23 3.08 3.48   
6   14,308.88 15,406.17 16,347.04                  4.57  4.67 4.09 4.38 5.03   
7   12,563.30 15,280.55 16,669.16                  3.60  3.48 3.24 3.58 3.21   
8   25,725.26 29,431.89 33,000.92                  7.16  6.31 6.45 6.44 6.55   

                                                    
Año 2015  2016  2017  2018  2019  2020  2021  2022  
0    NaN   NaN  8.35  9.22  9.84 11.46  8.74  6.80  
1   3.11  3.50  3.65  3.52  3.68  7.90  5.09  3.55  
2   8.54 11.58 12.79 12.33 11.94 13.70 13.16  9.23  
3   6.31  6.67  7.26  7.62  7.49 10.93  9.28  8.25  
4   8.57  8.92  9.09  9.36 10.28 15.98 13.90 10.55  
5   3.62  4.60  3.84  3.53  3.81  6.12  4.50  3.71  
6   4.56  5.25  6.14  6.22  6.59  7.55  7.31  6.75  
7   3.27  3.74  3.69  3.49  3.38  7.18  5.10  4.29  
8   7.49  7.84  7.89  8.34  8.84 10.41  9.33  7.88

En la siguiente tabla, vamos a mostrar, para cada país, el valor promedio del PIB per cápita PPA y de la tasa de desempleo entre 2010 y 2022.

In [79]:
df2 = pd.pivot_table(df, index = 'País', values = ['PIB per cápita (PPA, USD internacionales)',
                                                   'Tasa de desempleo (%)'],
                     aggfunc='mean').reset_index()
df2

,País,"PIB per cápita (PPA, USD internacionales)",Tasa de desempleo (%)
0,Argentina,"21,954.46",8.26
1,Bolivia,"7,650.02",3.57
2,Brazil,"15,822.21",10.16
3,Chile,"23,872.92",7.58
4,Colombia,"14,427.34",10.47
5,Ecuador,"11,586.79",3.93
6,Paraguay,"13,259.22",5.62
7,Peru,"12,396.53",3.94
8,Uruguay,"23,536.58",7.76


A continuación se muestra, para cada país, el valor promedio y la desviación estándar del PIB per cápita PPA y el máximo y mínimo de la tasa de desempleo en el período 2010-2022.

In [76]:
df3 = pd.pivot_table(df, index = 'País', values = ['PIB per cápita (PPA, USD internacionales)',
                                                   'Tasa de desempleo (%)'],
                     aggfunc={'PIB per cápita (PPA, USD internacionales)':['mean','std'],
                              'Tasa de desempleo (%)':['min', 'max']}).reset_index()
df3

País PIB per cápita (PPA, USD internacionales)           \
                                                  mean      std   
0  Argentina                                 21,954.46 3,381.37   
1    Bolivia                                  7,650.02 1,642.63   
2     Brazil                                 15,822.21 1,563.20   
3      Chile                                 23,872.92 3,483.97   
4   Colombia                                 14,427.34 2,644.02   
5    Ecuador                                 11,586.79 1,611.37   
6   Paraguay                                 13,259.22 1,672.52   
7       Peru                                 12,396.53 1,932.85   
8    Uruguay                                 23,536.58 4,243.44   

  Tasa de desempleo (%)       
                    max  min  
0                 11.46 6.80  
1                  7.90 2.02  
2                 13.70 6.75  
3                 10.93 6.07  
4                 15.98 8.57  
5                  6.12 3.08  
6                  7.55 4.09  
7                  7.18 3.21  
8                 10.41 6.31

## Transponer un dataframe

Dado que un dataframe es una matriz, lo podemos transponer.

In [85]:
# Dataframe inicial
df_desempleo

Año,País,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Argentina,7.71,7.18,7.22,7.10,7.27,NaN,NaN,8.35,9.22,9.84,11.46,8.74,6.80
1,Bolivia,NaN,2.25,2.07,2.44,2.02,3.11,3.50,3.65,3.52,3.68,7.90,5.09,3.55
2,Brazil,NaN,7.58,7.25,7.07,6.75,8.54,11.58,12.79,12.33,11.94,13.70,13.16,9.23
3,Chile,8.28,7.28,6.56,6.07,6.48,6.31,6.67,7.26,7.62,7.49,10.93,9.28,8.25
4,Colombia,11.15,10.29,9.96,9.25,8.80,8.57,8.92,9.09,9.36,10.28,15.98,13.90,10.55
5,Ecuador,4.09,3.46,3.23,3.08,3.48,3.62,4.60,3.84,3.53,3.81,6.12,4.50,3.71
6,Paraguay,4.57,4.67,4.09,4.38,5.03,4.56,5.25,6.14,6.22,6.59,7.55,7.31,6.75
7,Peru,3.60,3.48,3.24,3.58,3.21,3.27,3.74,3.69,3.49,3.38,7.18,5.10,4.29
8,Uruguay,7.16,6.31,6.45,6.44,6.55,7.49,7.84,7.89,8.34,8.84,10.41,9.33,7.88


In [86]:
# Dataframe transpuesto
df_desempleo_2 = df_desempleo.set_index('País').transpose().reset_index()
df_desempleo_2

País,Año,Argentina,Bolivia,Brazil,Chile,Colombia,Ecuador,Paraguay,Peru,Uruguay
0,2010,7.71,NaN,NaN,8.28,11.15,4.09,4.57,3.60,7.16
1,2011,7.18,2.25,7.58,7.28,10.29,3.46,4.67,3.48,6.31
2,2012,7.22,2.07,7.25,6.56,9.96,3.23,4.09,3.24,6.45
3,2013,7.10,2.44,7.07,6.07,9.25,3.08,4.38,3.58,6.44
4,2014,7.27,2.02,6.75,6.48,8.80,3.48,5.03,3.21,6.55
5,2015,NaN,3.11,8.54,6.31,8.57,3.62,4.56,3.27,7.49
6,2016,NaN,3.50,11.58,6.67,8.92,4.60,5.25,3.74,7.84
7,2017,8.35,3.65,12.79,7.26,9.09,3.84,6.14,3.69,7.89
8,2018,9.22,3.52,12.33,7.62,9.36,3.53,6.22,3.49,8.34
9,2019,9.84,3.68,11.94,7.49,10.28,3.81,6.59,3.38,8.84


## Reshape del dataframe

Transformando Formatos: de long a wide y de wide a long.


**Formato Largo (Long):** Cada observación está en su propia fila. (Ej: Tenemos columnas de País, Año, Indicador, Valor, por lo que cada fila es una observación). Es el formato ideal para almacenar datos.

**Formato Ancho (Wide):** Cada fila representa una unidad de observación (como un país) y las variables se extienden a lo largo de las columnas (Ej: Country, GDP_2010, GDP_2011, GDP_2012). Es ideal para analizar los datos.

### Pasar de wide a long

In [87]:
# Dataframe original
df_desempleo

Año,País,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Argentina,7.71,7.18,7.22,7.10,7.27,NaN,NaN,8.35,9.22,9.84,11.46,8.74,6.80
1,Bolivia,NaN,2.25,2.07,2.44,2.02,3.11,3.50,3.65,3.52,3.68,7.90,5.09,3.55
2,Brazil,NaN,7.58,7.25,7.07,6.75,8.54,11.58,12.79,12.33,11.94,13.70,13.16,9.23
3,Chile,8.28,7.28,6.56,6.07,6.48,6.31,6.67,7.26,7.62,7.49,10.93,9.28,8.25
4,Colombia,11.15,10.29,9.96,9.25,8.80,8.57,8.92,9.09,9.36,10.28,15.98,13.90,10.55
5,Ecuador,4.09,3.46,3.23,3.08,3.48,3.62,4.60,3.84,3.53,3.81,6.12,4.50,3.71
6,Paraguay,4.57,4.67,4.09,4.38,5.03,4.56,5.25,6.14,6.22,6.59,7.55,7.31,6.75
7,Peru,3.60,3.48,3.24,3.58,3.21,3.27,3.74,3.69,3.49,3.38,7.18,5.10,4.29
8,Uruguay,7.16,6.31,6.45,6.44,6.55,7.49,7.84,7.89,8.34,8.84,10.41,9.33,7.88


In [88]:
# Convertir a formato long
df_largo_desempleo = pd.melt(
    df_desempleo,
    id_vars=['País'], # Columnas que se mantienen fijas
    var_name='Año', # Nombre de la nueva columna de variables
    value_name='Desempleo (%)' # Nombre de la nueva columna de valores
)
df_largo_desempleo

,País,Año,Desempleo (%)
0,Argentina,2010,7.71
1,Bolivia,2010,NaN
2,Brazil,2010,NaN
3,Chile,2010,8.28
4,Colombia,2010,11.15
...,...,...,...
112,Colombia,2022,10.55
113,Ecuador,2022,3.71
114,Paraguay,2022,6.75
115,Peru,2022,4.29


El dataframe del Banco Mundial, que tiene forma de panel de datos y es wide, también lo podemos convertir a long.

In [89]:
df_largo = pd.melt(
    df,
    id_vars=['País', 'Año'], # Columnas que se mantienen fijas
    var_name='Indicador', # Nombre de la nueva columna de variables
    value_name='Valor' # Nombre de la nueva columna de valores
)
df_largo

,País,Año,Indicador,Valor
0,Argentina,2022,"PIB per cápita (PPA, USD internacionales)","29,597.69"
1,Argentina,2021,"PIB per cápita (PPA, USD internacionales)","26,300.27"
2,Argentina,2020,"PIB per cápita (PPA, USD internacionales)","22,393.35"
3,Argentina,2019,"PIB per cápita (PPA, USD internacionales)","23,516.83"
4,Argentina,2018,"PIB per cápita (PPA, USD internacionales)","24,410.39"
...,...,...,...,...
463,Uruguay,2014,Tasa de desempleo (%),6.55
464,Uruguay,2013,Tasa de desempleo (%),6.44
465,Uruguay,2012,Tasa de desempleo (%),6.45
466,Uruguay,2011,Tasa de desempleo (%),6.31


### Pasar de long a wide

Para pasar de formato long a wide se utilizan las tablas pivote que vimos previamente.

## Ejercicio

Utiliza las tablas pivote para obtener, a partir de *df_largo*, un dataframe en formato wide de la esperanza de vida (en cada fila hay un país y en cada columna van los años de 2010 a 2022). Luego transforma el dataframe para que los años queden en las filas y los países en las columnas.

In [94]:
df_esperanza = df.pivot_table(index='País', columns='Año', values='Esperanza de vida (años)').reset_index()
df_esperanza

Año,País,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Argentina,75.68,76.10,75.80,75.83,76.27,76.60,76.11,76.54,76.77,76.85,75.88,73.95,75.81
1,Bolivia,66.00,66.28,66.59,66.78,66.97,67.20,67.39,67.54,67.70,67.82,62.91,61.43,67.43
2,Brazil,73.78,74.05,74.33,74.61,74.82,75.11,75.08,75.38,75.63,75.81,74.51,73.04,74.87
3,Chile,79.09,79.24,79.22,79.55,79.71,80.01,80.30,80.61,80.56,80.32,79.35,78.88,79.18
4,Colombia,74.88,75.18,75.61,75.83,75.95,76.07,76.23,76.42,76.58,76.79,74.76,72.70,76.51
5,Ecuador,75.19,75.18,75.49,75.83,76.14,76.22,76.09,76.22,77.25,77.29,72.00,72.75,76.58
6,Paraguay,72.55,72.79,73.11,73.29,73.42,73.45,73.49,73.45,73.63,73.67,72.72,68.11,72.32
7,Peru,72.89,73.33,73.98,74.59,75.26,75.73,75.95,75.96,76.02,76.28,73.83,71.60,76.83
8,Uruguay,76.69,76.61,76.69,76.95,77.19,77.28,77.35,77.62,77.53,77.50,78.38,75.43,76.47
